In [ ]:
import sys
import random
import numpy as np
from scipy.spatial.distance import pdist, squareform

def read_xyz(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        header = lines[1:2]
        metal_atoms = [line.strip().split() for line in lines[2:] if line.startswith(atom_to_replace)]
        nonmetal_atoms = [line.strip().split() for line in lines[2:] if not line.startswith(atom_to_replace)]

    return metal_atoms, nonmetal_atoms, header

def write_xyz(file_path, metal_atoms, nonmetal_atoms, header):
    with open(file_path, 'w') as f:
        f.write(f"{len(metal_atoms) + len(nonmetal_atoms)}\n")
        f.write(''.join(header))
        for atom in metal_atoms:
            f.write(f"{' '.join(atom)}\n")
        for atom in nonmetal_atoms:
            f.write(f"{' '.join(atom)}\n")

def random_replace(metal_atoms, percentage):
    print('Generating random distribution ...')
    num_to_replace = int(len(metal_atoms) * percentage/100)
    indices = random.sample(range(len(metal_atoms)), num_to_replace)

    for i in indices:
        if metal_atoms[i][0] == atom_to_replace:
            metal_atoms[i][0] = atom_to_substitute
    
    print('Done!')
    return metal_atoms

def uniform_replace(metal_atoms, percentage):
    print('Generating uniform distribution ...')
    num_to_replace = int(len(metal_atoms) * percentage / 100)
    zn_indices = [i for i, atom in enumerate(metal_atoms) if atom[0] == atom_to_replace]
    zn_atoms = np.array([atom[1:] for atom in metal_atoms if atom[0] == atom_to_replace], dtype=float)
    distances = pdist(zn_atoms)
    dist_matrix = squareform(distances)
    indices = set()

    first_index = random.choice(zn_indices)
    indices.add(zn_indices.index(first_index))

    for _ in range(num_to_replace - 1):
        remaining_indices = set(range(len(zn_indices))) - indices
        nearest_neighbor_distances = np.min(dist_matrix[list(indices)][:, list(remaining_indices)], axis=0)
        index_to_add = np.argmax(nearest_neighbor_distances)
        indices.add(list(remaining_indices)[index_to_add])

    for index in indices:
        metal_atoms[zn_indices[index]][0] = atom_to_substitute

    print('Done!')
    return metal_atoms


def cluster_replace(metal_atoms, percentage):
    print('Generating clustered distribution ...')
    num_to_replace = int(len(metal_atoms) * percentage / 100)
    zn_indices = [i for i, atom in enumerate(metal_atoms) if atom[0] == atom_to_replace]
    zn_atoms = np.array([atom[1:] for atom in metal_atoms if atom[0] == atom_to_replace], dtype=float)
    distances = pdist(zn_atoms)
    dist_matrix = squareform(distances)
    indices = set()

    first_index = random.choice(zn_indices)
    indices.add(zn_indices.index(first_index))

    for _ in range(num_to_replace):
        mean_distances = np.mean(dist_matrix[list(indices)], axis=0)
        index_to_add = np.argmin(mean_distances)

        indices.add(zn_indices[index_to_add])
        dist_matrix[:, index_to_add] = float('inf')

    # print(len(indices))
    for index in indices:
        metal_atoms[zn_indices[index]][0] = atom_to_substitute

    print('Done!')
    return metal_atoms


def print_help():
    help_text = """
Usage: python script.py <Atom to be replaced> <Atom to substitute> <Percentage> <Mode>

Arguments:
    <Atom to be replaced>:   The atom symbol to be replaced (e.g., Zn)
    <Atom to substitute>:    The atom symbol to substitute (e.g., Mg)
    <Percentage>:            The percentage of atoms to replace (e.g., 25)
    <Mode>:                  The replacement mode (1, 2, or 3)

Modes:
    1 - Random replacement
    2 - Uniform replacement
    3 - Clustered replacement
    """
    print(help_text)


if __name__ == "__main__":
    if len(sys.argv) != 5 or sys.argv[1] in ["--help", "-h"]:
        print_help()
        sys.exit(1)

    atom_to_replace = sys.argv[1]
    atom_to_substitute = sys.argv[2]
    percentage = float(sys.argv[3])
    mode = int(sys.argv[4])

    metal_atoms, nonmetal_atoms, header = read_xyz('xmolout')
    metal_atoms = [atom for atom in metal_atoms if atom[0] == atom_to_replace]

    if mode == 1:
        metal_atoms = random_replace(metal_atoms, percentage)
        write_xyz('random.xyz', metal_atoms, nonmetal_atoms, header)
    elif mode == 2:
        metal_atoms = uniform_replace(metal_atoms, percentage)
        write_xyz('uniform.xyz', metal_atoms, nonmetal_atoms, header)
    elif mode == 3:
        metal_atoms = cluster_replace(metal_atoms, percentage)
        write_xyz('clustered.xyz', metal_atoms, nonmetal_atoms, header)
    else:
        print("Invalid mode. Choose 1, 2, or 3.")
        sys.exit(1)